<a href="https://colab.research.google.com/github/yadgire7/Machine-Learning/blob/master/Support_Vector_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Saurabh Arun Yadgire USC ID: 8065527198 Github ID:yadgire7

Resources:
1. https://towardsdatascience.com/evaluating-multi-label-classifiers-a31be83da6ea
2. https://stackoverflow.com/questions/32239577/getting-the-accuracy-for-multi-label-prediction-in-scikit-learn
3. http://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html
4. https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
5. https://towardsdatascience.com/k-means-clustering-from-a-to-z-f6242a314e9a
6. https://stanford.edu/~cpiech/cs221/handouts/kmeans.html
7.



# Import Libraries

In [ ]:
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedShuffleSplit,KFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.metrics import accuracy_score,classification_report, silhouette_samples,silhouette_score,hamming_loss
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
import statistics
import random

# Load Data

In [ ]:
data = pd.read_csv(r'../Data/Frogs_MFCCs.csv')
x = data.iloc[:,:-4]
y = data.loc[:,['Family','Genus','Species']]
print(y.columns)
# print(y)
print(data)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,shuffle=True)
# print(x_train)


Index(['Family', 'Genus', 'Species'], dtype='object')
      MFCCs_ 1  MFCCs_ 2  MFCCs_ 3  MFCCs_ 4  MFCCs_ 5  MFCCs_ 6  MFCCs_ 7  \
0          1.0  0.152936 -0.105586  0.200722  0.317201  0.260764  0.100945   
1          1.0  0.171534 -0.098975  0.268425  0.338672  0.268353  0.060835   
2          1.0  0.152317 -0.082973  0.287128  0.276014  0.189867  0.008714   
3          1.0  0.224392  0.118985  0.329432  0.372088  0.361005  0.015501   
4          1.0  0.087817 -0.068345  0.306967  0.330923  0.249144  0.006884   
...        ...       ...       ...       ...       ...       ...       ...   
7190       1.0 -0.554504 -0.337717  0.035533  0.034511  0.443451  0.093889   
7191       1.0 -0.517273 -0.370574  0.030673  0.068097  0.402890  0.096628   
7192       1.0 -0.582557 -0.343237  0.029468  0.064179  0.385596  0.114905   
7193       1.0 -0.519497 -0.307553 -0.004922  0.072865  0.377131  0.086866   
7194       1.0 -0.508833 -0.324106  0.062068  0.078211  0.397188  0.094596   

      MFC

## Q.1.b.

In [ ]:
print(data['Family'].values)
print(data['Genus'].values)
print(data['Species'].values)
# It can be observed that each label is binary

['Leptodactylidae' 'Leptodactylidae' 'Leptodactylidae' ... 'Hylidae'
 'Hylidae' 'Hylidae']
['Adenomera' 'Adenomera' 'Adenomera' ... 'Scinax' 'Scinax' 'Scinax']
['AdenomeraAndre' 'AdenomeraAndre' 'AdenomeraAndre' ... 'ScinaxRuber'
 'ScinaxRuber' 'ScinaxRuber']


## Q.1.b.i.

Techniques used to measure a multi-class classifier:
1. micro-averaging
2. Macro averaging
3. Hamming Loss:
    - Fraction of labels that are incorrectly predicted (mis-labelling)
4. Exact Match Ratio (subset accuracy):
    - Most strict metric
    - indicates percentage that have all labels classified correctly.
    - ignores partially correct matches
    - accuracy_score from scikit-learn

Multi-label classification techniques:
- One vs rest:
    - build multiple independent classifiers and assign the class to the test data which has maximum confidence.
    - ASSUMPTION : Labels are mutually exclusive i.e. Family, genus and Species are mutually exclusive (not correlated)
    -

## Q.1.b.ii.

In [ ]:
# #def hamming_score(y_true, y_pred):
#     return (
#         (y_true & y_pred).sum(axis=1) / (y_true | y_pred).sum(axis=1)
#     ).mean()


# hamming_score(y_true, y_pred)

In [ ]:
SVC_pipeline = Pipeline([('clf', OneVsRestClassifier(SVC(gamma='auto'), n_jobs=-1)),])
for c in y_train.columns:
    SVC_pipeline.fit(x_train,y_train[c])
    pred = SVC_pipeline.predict(x_test)
    print(f'Test accuracy for considering {c} as target class is {accuracy_score(y_test[c],pred)}\n')

Test accuracy for considering Family as target class is 0.9314497452524316

Test accuracy for considering Genus as target class is 0.9059749884205651

Test accuracy for considering Species as target class is 0.9143121815655396



gamma and c paramaters of Radial Basis Function (RBF) kernel
c parameter is the regularization(tuning) paramter of SVM
gamma parameter defines the reach/influence of each training point. Large value of gamma means 'close' and small values of gamma mean 'far'. It can be seen as inverse of the radius of influence.

In [ ]:
warnings.filterwarnings(action='ignore')
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
C_range = np.logspace(-2, 2, 5)
gamma_range = np.logspace(-2, 2, 5)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = KFold(n_splits=10)
for c in y_train.columns:
    grid = GridSearchCV(SVC(kernel='rbf',decision_function_shape='ovr'),param_grid=param_grid,cv=cv)
    grid.fit(x_train,y_train[c])
    print(f'The best parameters considering {c} as the target label are: %s with a score of %0.2f'%(grid.best_params_,grid.best_score_))

The best parameters considering Family as the target label are: {'C': 10.0, 'gamma': 0.1} with a score of 0.99
The best parameters considering Genus as the target label are: {'C': 100.0, 'gamma': 0.01} with a score of 0.99
The best parameters considering Species as the target label are: {'C': 100.0, 'gamma': 0.01} with a score of 0.99


## Q1.b.iii.

# K-Fold Cross Validation

In [ ]:
warnings.filterwarnings(action='ignore')
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)   #data has already been standardized
C_range = np.logspace(-2, 2, 5)
params = dict(C=C_range)
cv = KFold(n_splits=10)
for c in y_train.columns:
  g = GridSearchCV(LinearSVC(penalty='l1',loss='squared_hinge',multi_class='ovr',dual=False),param_grid=params,cv=cv)
  g.fit(x_train,y_train[c])
  print(f'The best parameters considering {c} as the target label are: %s with a score of %0.2f'%(g.best_params_,g.best_score_))

The best parameters considering Family as the target label are: {'C': 1.0} with a score of 0.94
The best parameters considering Genus as the target label are: {'C': 100.0} with a score of 0.95
The best parameters considering Species as the target label are: {'C': 100.0} with a score of 0.96


In [ ]:
oversample = SMOTE()
warnings.filterwarnings(action='ignore')
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)   #data has already been standardized
C_range = np.logspace(-2, 2, 5)
params = dict(C=C_range)
cv = KFold(n_splits=10)
for c in y_train.columns:
  x_train_resample, y_train_resample = oversample.fit_resample(x_train,y_train[c])
  g_smote = GridSearchCV(LinearSVC(penalty='l1',loss='squared_hinge',multi_class='ovr',dual=False),param_grid=params,cv=cv)
  g_smote.fit(x_train_resample,y_train_resample)
  print(f'The best parameters considering {c} as the target label are: %s with a score of %0.2f'%(g_smote.best_params_,g_smote.best_score_))

The best parameters considering Family as the target label are: {'C': 100.0} with a score of 0.94
The best parameters considering Genus as the target label are: {'C': 10.0} with a score of 0.92
The best parameters considering Species as the target label are: {'C': 100.0} with a score of 0.90


## Q.2.a.

In [ ]:
family={p:[] for p in range(1,51)}
genus={p:[] for p in range(1,51)}
species={p:[] for p in range(1,51)}
random_int = random.randint(0,700)
ham_loss = []
ham_score = []



In [ ]:
for c in range(1,51):
    k_sil_dict = dict()
    for k in range(2,20):
        kmeans = KMeans(n_clusters=k,random_state=random_int).fit(x)
        pred_labels = kmeans.labels_
    #     print(pred_labels)
        avg_sil_score = silhouette_score(x,pred_labels)
        k_sil_dict.update({k:avg_sil_score})
    # print(k_sil_dict)
    best_k = max(k_sil_dict,key=k_sil_dict.get)
    temp_df = x
    kmeans_best = KMeans(n_clusters=best_k,random_state=random_int).fit(temp_df)
    best_labels = kmeans_best.labels_
    # print(best_labels)
    cluster_df = pd.concat([temp_df,y,pd.DataFrame({'label':best_labels.tolist()})],axis=1)
    cluster_df['label'].value_counts()

    # print(temp_df.shape[1])
    # print(len(temp_df.columns))
#     print(cluster_df['label'].value_counts())
    # # print(best_k)
    for k in range(4):
        filter_df = cluster_df[cluster_df['label']==k]
    triplet = {k:[] for k in range(4)}
    for k in range(4):
        clus_val = cluster_df[cluster_df['label']==k]
        triplet[k].append(clus_val['Family'].value_counts().index[0])
        triplet[k].append(clus_val['Genus'].value_counts().index[0])
        triplet[k].append(clus_val['Species'].value_counts().index[0])
        family[c].append(clus_val['Family'].value_counts().index[0])
        genus[c].append(clus_val['Family'].value_counts().index[0])
        species[c].append(clus_val['Family'].value_counts().index[0])
    cluster_df['Family_Pred'] = 'none'
    cluster_df['Genus_Pred'] = 'none'
    cluster_df['Species_Pred'] = 'none'
    for k_dash in range(4):
        cluster_df['Family_Pred'] = np.where(cluster_df['label']==k,triplet[k][0],cluster_df['Family_Pred'])
        cluster_df['Genus_Pred'] = np.where(cluster_df['label']==k,triplet[k][0],cluster_df['Genus_Pred'])
        cluster_df['Species_Pred'] = np.where(cluster_df['label']==k,triplet[k][0],cluster_df['Species_Pred'])
    family_hl = hamming_loss(cluster_df['Family'],cluster_df['Family_Pred'])
    genus_hl = hamming_loss(cluster_df['Genus'],cluster_df['Genus_Pred'])
    species_hl = hamming_loss(cluster_df['Species'],cluster_df['Species_Pred'])
    avg_hs = (family_hl+genus_hl+species_hl)/3
    ham_loss.append(np.round(avg_hs,5))
    ham_score.append(1-avg_hs)
print(f'The standaed deviation of all the 50 Hamming distances is: {statistics.stdev(ham_score)}')
print(f'The mean of all the 50 Hamming distances is: {statistics.mean(ham_score)}')
# print(triplet)

The standaed deviation of all the 50 Hamming distances is: 0.0
The mean of all the 50 Hamming distances is: 0.0231642344220524


In [ ]:
final_list = list(zip(list(family.values()),list(genus.values()),list(species.values())))
# print(final_list)

final_triplet={}
for i in range(0,50):
    final_triplet.update({i:list(zip(final_list[i][0],final_list[i][1],final_list[i][2]))})


In [ ]:
for i in range(0,50):
    print(f'Interation = {i+1}, best K = {best_k}, Hamming Score = {ham_score[i]}, Best Triplet : {final_triplet[i]}\n')

Interation = 1, best K = 4, Hamming Score = 0.0231642344220524, Best Triplet : [('Leptodactylidae', 'Leptodactylidae', 'Leptodactylidae'), ('Hylidae', 'Hylidae', 'Hylidae'), ('Hylidae', 'Hylidae', 'Hylidae'), ('Dendrobatidae', 'Dendrobatidae', 'Dendrobatidae'), ('Leptodactylidae', 'Leptodactylidae', 'Leptodactylidae'), ('Hylidae', 'Hylidae', 'Hylidae'), ('Hylidae', 'Hylidae', 'Hylidae'), ('Dendrobatidae', 'Dendrobatidae', 'Dendrobatidae')]

Interation = 2, best K = 4, Hamming Score = 0.0231642344220524, Best Triplet : [('Leptodactylidae', 'Leptodactylidae', 'Leptodactylidae'), ('Hylidae', 'Hylidae', 'Hylidae'), ('Hylidae', 'Hylidae', 'Hylidae'), ('Dendrobatidae', 'Dendrobatidae', 'Dendrobatidae')]

Interation = 3, best K = 4, Hamming Score = 0.0231642344220524, Best Triplet : [('Leptodactylidae', 'Leptodactylidae', 'Leptodactylidae'), ('Hylidae', 'Hylidae', 'Hylidae'), ('Hylidae', 'Hylidae', 'Hylidae'), ('Dendrobatidae', 'Dendrobatidae', 'Dendrobatidae')]

Interation = 4, best K = 4, H